## In this code I am gonna  to perform the parameters exploration using Planck 2018 TT (Planck collaboration 2018) Cl data  together with Cobaya sampling (Torrado & Lewis, 2022) and GetDist

In [50]:
# Importing libraries
# Configuration
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import camb
import sys, platform, os
try:
    from cobaya import run
except ImportError:
    sys.path.insert(0,'../../cobaya')
    from cobaya import run
import numpy as np
import matplotlib.pyplot as plt
import getdist
from getdist import plots, MCSamples
import healpy as hp

Load the Planck data

Ploting the spectra

We compute the theoretical cls with Camb to make a cmb map

In [46]:
# Hypothetical function to compute the theoretical power spectrum
def theoretical_Cls(H0, ombh2, omch2, mnu, omk, tau, As, ns, lmax):
    # Set up a new set of parameters for CAMB
    pars = camb.set_params(H0=H0, ombh2=ombh2, omch2=omch2, mnu=mnu, omk=omk, tau=tau,  
                           As=As, ns=ns, lmax=lmax)
    # Ensure lmax is an integer
    lmax = int(lmax)
    print(pars)
    # Calculate results for these parameters
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    totCL = powers['total']  # Get the total power spectrum (including lensed scalar)

    # Extract the theoretical power spectrum (Dl_th)
    Dl_th = totCL[:, 0]  # TT spectrum is the first column

    # Get the corresponding ell values
    ells = np.arange(0, lmax + 1)
    
    return ells, Dl_th
# Define cosmological parameters and lmax
lmax = 2000
H0 = 67.4
ombh2 = 0.0224
omch2 = 0.120
mnu = 0.06
omk = 0.0
tau =  0.054
As = 2e-9
ns = 0.965

# Compute the theoretical power spectrum
ells, Dl_th = theoretical_Cls(H0, ombh2, omch2, mnu, omk, tau, As, ns, lmax)

class: <CAMBparams>
 WantCls = True
 WantTransfer = False
 WantScalars = True
 WantTensors = False
 WantVectors = False
 WantDerivedParameters = True
 Want_cl_2D_array = True
 Want_CMB = True
 Want_CMB_lensing = True
 DoLensing = True
 NonLinear = NonLinear_none
 Transfer: <TransferParams>
   high_precision = False
   accurate_massive_neutrinos = False
   kmax = 0.9
   k_per_logint = 0
   PK_num_redshifts = 1
   PK_redshifts = [0.0]
 want_zstar = False
 want_zdrag = False
 min_l = 2
 max_l = 2150
 max_l_tensor = 600
 max_eta_k = 5375.0
 max_eta_k_tensor = 1200.0
 ombh2 = 0.0224
 omch2 = 0.12
 omk = 0.0
 omnuh2 = 0.000644866570625114
 H0 = 67.4
 TCMB = 2.7255
 YHe = 0.24586829360147697
 num_nu_massless = 2.0293333333333337
 num_nu_massive = 1
 nu_mass_eigenstates = 1
 share_delta_neff = False
 nu_mass_degeneracies = [1.0146666666666666]
 nu_mass_fractions = [1.0]
 nu_mass_numbers = [1]
 InitPower: <InitialPowerLaw>
   tensor_parameterization = tensor_param_rpivot
   ns = 0.965
   nrun =

# Generate a CMB map realization

In [53]:
seed=583
np.random.seed(seed)
nside=1024

In [54]:
alm=hp.synalm((Dl_th), lmax=lmax)

In [55]:
cmb_map=hp.alm2map(alm, nside=nside, lmax=lmax)

In [62]:
cl = hp.anafast(cmb_map, lmax=lmax, use_pixel_weights=True) * 1e12 # in muK^2
ell = np.arange(len(cl))
cl *= ell * (ell+1) / (2*np.pi)

In [64]:
cl.shape

(2001,)

In [65]:
plt.plot(ell, cl, '--')
plt.plot(ells, Dl_th)
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\dfrac{\ell(\ell+1)}{2\pi} C_\ell~[\mu K^2]$")
plt.xlim([2, 2500])
plt.show()


TypeError: 'tuple' object is not callable